In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
import json
import pandas as pd
import io
import datetime

In [12]:
params = {"cond": "covid", "down_count": 100, "down_fmt": "csv"}
# r = requests.get('https://clinicaltrials.gov/ct2/results/', params)
r = requests.get('https://clinicaltrials.gov/ct2/results/download_fields?cond=COVID-19&down_count=1000&down_fmt=csv')
                  
print(r.status_code)

200


In [14]:
df = pd.read_csv(io.BytesIO(r.content))
df.head(100)

,Rank,Title,Status,Study Results,Conditions,Interventions,Locations,URL
0,1,Duvelisib to Combat COVID-19,Not yet recruiting,No Results Available,COVID-19,Drug: Duvelisib|Procedure: Peripheral blood draw,"Washington University School of Medicine, Sain...",https://ClinicalTrials.gov/show/NCT04372602
1,2,Observational Cohort of COVID-19 Patients at R...,Recruiting,No Results Available,COVID-19,NaN,"Department of Infectiology, Raymond Poincaré H...",https://ClinicalTrials.gov/show/NCT04364698
2,3,A Phase 2 Trial of Infliximab in Coronavirus D...,Recruiting,No Results Available,COVID-19,Drug: Infliximab,"Tufts Medical Center, Boston, Massachusetts, U...",https://ClinicalTrials.gov/show/NCT04425538
3,4,Safety and Efficacy of C21 in Subjects With CO...,Recruiting,No Results Available,COVID-19,Drug: C21|Drug: Placebo,"Department of Medicine, Civil Hospital and B J...",https://ClinicalTrials.gov/show/NCT04452435
4,5,Application of Desferal to Treat COVID-19,Recruiting,No Results Available,COVID-19,Drug: Deferoxamine,"Regenerative Medicine Research Center, Kermans...",https://ClinicalTrials.gov/show/NCT04333550
...,...,...,...,...,...,...,...,...
95,96,Favipiravir in Hospitalized COVID-19 Patients,Not yet recruiting,No Results Available,COVID-19,Drug: Favipiravir|Drug: Hydroxychloroquine,"Shahid Modarres Hospital, Shahid Beheshti Univ...",https://ClinicalTrials.gov/show/NCT04359615
96,97,Treatment of Moderate to Severe Coronavirus Di...,Recruiting,No Results Available,COVID-19,Drug: Lopinavir/ritonavir|Drug: Hydroxychloroq...,"Nova Scotia Health Authority, Halifax, Nova Sc...",https://ClinicalTrials.gov/show/NCT04321993
97,98,Azithromycin in Hospitalized COVID-19 Patients,Not yet recruiting,No Results Available,COVID-19,Drug: Hydroxychloroquine|Drug: Azithromycin,"Shahid Modarres Hospital, Shahid Beheshti Univ...",https://ClinicalTrials.gov/show/NCT04359316
98,99,Thymosin Alpha 1 to Prevent COVID-19 Infection...,Not yet recruiting,No Results Available,COVID-19,Drug: Thymalfasin,NaN,https://ClinicalTrials.gov/show/NCT04428008


In [6]:
CDC_BASE_URL = 'https://clinicaltrials.gov/api/query/study_fields?expr=COVID-19&max_rnk=1000&fmt=json'

### Full list of allowable fields can be found at 
https://clinicaltrials.gov/api/info/study_fields_list

In [16]:
cdc_extract_fields = [
    'BriefTitle',
    'Phase',
    'DesignAllocation',
    'DesignMasking',
    'DesignMaskingDescription',
    'InterventionName',
    'InterventionType',
    'LastKnownStatus',
    'OfficialTitle',
    'OutcomeAnalysisStatisticalMethod',
    'OutcomeMeasureTimeFrame',
    'SecondaryOutcomeMeasure',
    'StartDate',
    'StudyFirstPostDate',
    'StudyFirstPostDateType',
    'StudyFirstSubmitDate',
    'StudyFirstSubmitQCDate',
    'StudyPopulation',
    'StudyType',
    'WhyStopped'
]

query_url = f'{CDC_BASE_URL}&fields={",".join(cdc_extract_fields)}'
print(query_url)

https://clinicaltrials.gov/api/query/study_fields?expr=COVID-19&max_rnk=1000&fmt=json&fields=BriefTitle,Phase,DesignAllocation,DesignMasking,DesignMaskingDescription,InterventionName,InterventionType,LastKnownStatus,OfficialTitle,OutcomeAnalysisStatisticalMethod,OutcomeMeasureTimeFrame,SecondaryOutcomeMeasure,StartDate,StudyFirstPostDate,StudyFirstPostDateType,StudyFirstSubmitDate,StudyFirstSubmitQCDate,StudyPopulation,StudyType,WhyStopped


In [17]:
r = requests.get(query_url)
print(r.status_code)

200


In [18]:
j = json.loads(r.content)
df = pd.DataFrame(j['StudyFieldsResponse']['StudyFields'])

In [19]:
def de_list(input_field):
    if isinstance(input_field, list):
        if len(input_field) == 0:
            return None
        elif len(input_field) == 1:
            return input_field[0]
        else:
            return '; '.join(input_field)
    else:
        return input_field

In [20]:
for c in df.columns:
    df[c] = df[c].apply(de_list)

In [21]:
df['StudyFirstPostDate'] = pd.to_datetime(df.StudyFirstPostDate)
df = df.sort_values(by='StudyFirstPostDate', ascending=False)

In [22]:
df[df.StudyType == 'Interventional'].head(100)

,Rank,BriefTitle,Phase,DesignAllocation,DesignMasking,DesignMaskingDescription,InterventionName,InterventionType,LastKnownStatus,OfficialTitle,...,OutcomeMeasureTimeFrame,SecondaryOutcomeMeasure,StartDate,StudyFirstPostDate,StudyFirstPostDateType,StudyFirstSubmitDate,StudyFirstSubmitQCDate,StudyPopulation,StudyType,WhyStopped
860,861,Study of the Efficacy and Safety of a Single A...,Phase 2; Phase 3,Randomized,Double,None,Olokizumab 64 mg; Placebo,Drug; Drug,None,"A Multicenter, Randomized, Double-blind, Adapt...",...,None,Subjects' clinical status distribution based o...,"June 30, 2020",2020-06-30,Estimate,"June 25, 2020","June 26, 2020",None,Interventional,None
264,265,Probe-based Confocal Laser Endomicroscopy in C...,Not Applicable,Non-Randomized,None (Open Label),None,miniprobe Alveoflex,Device,None,Probe-based Confocal Laser Endomicroscopy in C...,...,None,Number of Participants With the Correspondence...,"July 1, 2020",2020-06-30,Estimate,"June 29, 2020","June 29, 2020",None,Interventional,None
974,975,Statistical and Epidemiological Study Based on...,Phase 1; Phase 2,Randomized,Double,None,Convalescent plasma,Biological,None,"Pilot Clinical, Statistical and Epidemiologica...",...,None,Length of stay in Intensive Care Unit (ICU); L...,"July 6, 2020",2020-06-30,Estimate,"June 29, 2020","June 29, 2020",None,Interventional,None
475,476,Safety and Efficacy of C21 in Subjects With CO...,Phase 2,Randomized,Triple,None,C21; Placebo,Drug; Drug,None,"A Randomised, Double-blind, Placebo-controlled...",...,None,None,June 2020,2020-06-30,Estimate,"June 29, 2020","June 29, 2020",None,Interventional,None
728,729,Study of Decidual Stromal Cells to Treat COVID...,Not Applicable,N/A,None (Open Label),None,Decidual Stromal Cells (DSC),Biological,None,DSC-COVID-19: An Open-label Study on the Safet...,...,None,Mortality rate from COVID-19; Mortality rate f...,"July 6, 2020",2020-06-30,Estimate,"June 26, 2020","June 26, 2020",None,Interventional,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,640,A Trial of CONvalescent Plasma for Hospitalize...,Phase 3,Randomized,None (Open Label),None,Convalescent plasma,Biological,None,CONCOR-1: A Randomized Open-Label Trial of CON...,...,None,Need for Intubation; Time to intubation; Venti...,June 2020,2020-06-05,Actual,"June 3, 2020","June 4, 2020",None,Interventional,None
348,349,Treatment of Patients With COVID-19 With Conva...,Phase 2,Randomized,None (Open Label),None,convalescent plasma,Biological,None,Treatment of Patients With COVID-19 With Conva...,...,None,acute adverse events; Clinical Status; Duratio...,"June 1, 2020",2020-06-04,Actual,"June 1, 2020","June 2, 2020",None,Interventional,None
346,347,Bacillus Calmette-guérin Vaccination to Preven...,Phase 4,Randomized,Quadruple,None,BCG vaccine; Placebo,Biological; Biological,None,A Randomized Clinical Trial for Enhanced Train...,...,None,Positive respiratory questionnaire endpoint co...,"May 26, 2020",2020-06-04,Actual,"June 1, 2020","June 1, 2020",None,Interventional,None
236,237,Efficiency and Security of NIVOLUMAB Therapy i...,Phase 2,Randomized,None (Open Label),None,NIVOLUMAB; Routine standard of care,Drug; Other,None,Study of the Efficiency and Security of NIVOLU...,...,None,Readmission; Mortality; Oxygen flow needs; Req...,"June 15, 2020",2020-06-04,Actual,"June 2, 2020","June 2, 2020",None,Interventional,None
